In [29]:
%reload_ext autoreload
%autoreload 2
if '__file__' not in globals():
    __file__, __name__ = globals()['__vsc_ipynb_file__'], '__ipynb__'
    import types, sys; sys.modules['__ipynb__'] = types.ModuleType('__ipynb__')

import sys, os
if os.path.abspath('.') not in sys.path: sys.path.append(os.path.abspath('.'))

from pathlib import Path
from glob import glob

class text_color:
    def __init__(self, *attrs): self.clrs = list(set(attrs))
    def __ror__(self, obj): return f'\33[{";".join(map(str, self.clrs))}m{str(obj)}\33[0m'
    @property
    def bg(self): return text_color( next((c+70 for c in self.clrs if 30 <= c < 38), self.clrs[0]))

bold, italic, underline, strike = [1,3,4,9]
black,red,green,yellow,blue,magenta,cyan,white = [text_color(clr) for clr in range(30,38)]

import inspect
class cout:
    def __ror__(self, obj): print(f'[{inspect.stack()[1].lineno}]'|green, obj)
    def __call__(self, *args, **kwds): print(f'[{inspect.stack()[1].lineno}]'|green, *args, **kwds)
out = cout()

from dotenv import load_dotenv
load_dotenv()

jjkim_key = os.getenv("JINA_API_JJKIM")
obj76_key = os.getenv("JINA_API_OBJECTS76")

from rich import print as pprint
from IPython.display import HTML, Markdown

In [ ]:
import shlex
import subprocess
import locale

def run_web_llm(profile: str, url: str) -> None:
    BROWSER = "microsoft-edge-stable"

    # Sanitize inputs to prevent command injection
    safe_user = shlex.quote(profile)
    safe_url = shlex.quote(url)
    args = [
        BROWSER,
        f"--profile-directory={safe_user}",
        # "--new-tab",
        safe_url
    ]

    # Launch browser detached (non-blocking)
    subprocess.Popen(args,
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.DEVNULL,
                    stdin=subprocess.DEVNULL,
                    start_new_session=True  # Detach from parent process
                    )
    print(f"Browser launched: {' '.join(args)}")

run_web_llm("Default", "https://www.chatgpt.com")

Browser launched: microsoft-edge-stable --profile-directory=Default https://www.chatgpt.com


In [ ]:
import time
import subprocess

def wait_page_loaded(needle, timeout=3):
    cmds = "xdotool search --name --class microsoft-edge"

    result = subprocess.run(cmds.split(), capture_output=True, text=True, timeout=timeout)
    window_ids = [w for w in result.stdout.strip().split('\n') if w]

    for window_id in window_ids:
        print('activate: ', window_id)
        result = subprocess.run(['xdotool', 'windowactivate', window_id], capture_output=True, text=True, check=True, timeout=timeout)
        if result.returncode == 0 and not result.stderr.strip():
            print('windowactivate: ', result)

            result = subprocess.run(['xdotool', 'getwindowname', window_id], capture_output=True, text=True, check=True, timeout=timeout)
            if result.returncode == 0:
                title = result.stdout.strip()
                print('getwindowname: ', title)
                if needle.lower() in title.lower():
                    print('page loaded: ', title)
                    return True
    return False


wait_page_loaded('chatgpt')


['54525953', '44040196', '54525955']
activate:  54525953
activate:  44040196
windowactivate:  CompletedProcess(args=['xdotool', 'windowactivate', '44040196'], returncode=0, stdout='', stderr='')
getwindowname:  ChatGPT and 3 more pages - Personal - Microsoft​ Edge
page loaded:  ChatGPT and 3 more pages - Personal - Microsoft​ Edge


True

In [ ]:
import time
import subprocess
import re

def wait_for_chatgpt_loaded(timeout=30):
    """
    Wait for ChatGPT to load in the detached browser process
    """
    start_time = time.time()

    while time.time() - start_time < timeout:
        try:
            # 1. First check if Edge process is running
            result = subprocess.run(['pgrep', '-f', 'microsoft-edge'],
                                    capture_output=True, text=True)
            if not result.stdout.strip():
                print("❌ Edge process not found")
                time.sleep(1)
                continue

            # 2. Find Edge window
            result = subprocess.run(['xdotool', 'search', '--name', '--class', 'microsoft-edge'],
                                    capture_output=True, text=True)
            window_ids = [w for w in result.stdout.strip().split('\n') if w]

            if not window_ids:
                print("⏳ Waiting for Edge window...")
                time.sleep(1)
                continue

            window_id = window_ids[0]

            # 3. Check window title contains ChatGPT
            result = subprocess.run(['xdotool', 'getwindowname', window_id],
                                    capture_output=True, text=True)
            window_title = result.stdout.strip()

            if 'ChatGPT' not in window_title and 'chatgpt' not in window_title.lower():
                print(f"⏳ Waiting for ChatGPT page... (current: {window_title[:50]})")
                time.sleep(2)
                continue

            # 4. Try to interact with the page (test if it's responsive)
            try:
                subprocess.run(['xdotool', 'windowactivate', window_id], check=True, timeout=2)
                time.sleep(0.3)

                # 5. Check if we can find the chat input (try to focus it)
                # ChatGPT typically has a textarea for input
                subprocess.run(['xdotool', 'key', 'ctrl+l'], check=True)  # Focus address bar
                time.sleep(0.1)
                subprocess.run(['xdotool', 'key', 'Tab'], check=True)     # Tab to page content

                print("✅ ChatGPT page is interactive!")
                return True

            except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
                print("⏳ Page not yet interactive...")
                time.sleep(1)
                continue

        except subprocess.CalledProcessError as e:
            print(f"⏳ Checking... ({int(time.time() - start_time)}s)")
            time.sleep(2)

    print("❌ Timeout waiting for ChatGPT to load")
    return False

# Alternative: Check page URL in browser
def check_chatgpt_url_loaded(timeout=20):
    """
    Alternative method: Check if browser URL contains chatgpt
    """
    start_time = time.time()

    while time.time() - start_time < timeout:
        try:
            # Get active window URL (works with some browsers)
            result = subprocess.run([
                'xdotool', 'search', '--name', '--class', 'microsoft-edge'
            ], capture_output=True, text=True)

            if result.stdout.strip():
                window_id = result.stdout.strip().split('\n')[0]

                # Activate window and get URL from address bar
                subprocess.run(['xdotool', 'windowactivate', window_id], check=True)
                time.sleep(0.2)
                subprocess.run(['xdotool', 'key', 'ctrl+l'], check=True)  # Focus address bar
                time.sleep(0.1)
                subprocess.run(['xdotool', 'key', 'ctrl+a'], check=True)  # Select all
                subprocess.run(['xdotool', 'key', 'ctrl+c'], check=True)  # Copy URL

                # Get clipboard content (URL)
                clipboard_result = subprocess.run(['xclip', '-o', '-selection', 'clipboard'],
                                                capture_output=True, text=True)
                url = clipboard_result.stdout.strip()

                if 'chatgpt.com' in url.lower():
                    print(f"✅ ChatGPT URL loaded: {url}")
                    return True
                else:
                    print(f"⏳ Current URL: {url}")

        except (subprocess.CalledProcessError, subprocess.TimeoutExpired):
            pass

        time.sleep(1)

    return False

# Update the launch function to include wait
def run_web_llm_with_wait(profile: str, url: str):
    import shlex

    BROWSER = "microsoft-edge-stable"
    safe_user = shlex.quote(profile)
    safe_url = shlex.quote(url)
    args = [BROWSER, f"--profile-directory={safe_user}", safe_url]

    # Launch browser detached
    subprocess.Popen(args,
                    stdout=subprocess.DEVNULL,
                    stderr=subprocess.DEVNULL,
                    stdin=subprocess.DEVNULL,
                    start_new_session=True)

    print(f"🚀 Browser launched: {' '.join(args)}")

    # Wait for ChatGPT to load
    if wait_for_chatgpt_loaded(30):
        print("🎉 ChatGPT is ready for interaction!")
        return True
    else:
        print("⚠️ ChatGPT may not be fully loaded")
        return False

# Test it
result = run_web_llm_with_wait("Default", "https://www.chatgpt.com")
print(f"Load result: {result}")

🚀 Browser launched: microsoft-edge-stable --profile-directory=Default https://www.chatgpt.com
⏳ Waiting for ChatGPT page... (current: 15 Python Tricks That Separated Me from Average De)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-stable)
⏳ Waiting for ChatGPT page... (current: microsoft-edge-sta